In [1]:
from google import genai
import os
from dotenv import load_dotenv
import json


In [2]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
client = genai.Client(api_key=gemini_api_key)

In [4]:
# Load the training dataset using relative path
with open("../data/training.json", "r") as f:
    data = json.load(f)

messages = data["messages"]

# Show a few samples
for msg in messages[:5]:
    print(f"Text: {msg['message']}\nCategory: {msg['category']}\n")

Text: I want to place an order for two T-shirts.
Category: new_order

Text: Do you have the black shoes in size 10?
Category: new_order

Text: Can I buy 5 units of your protein powder?
Category: new_order

Text: I’m interested in your skincare bundle. How can I order?
Category: new_order

Text: Is the winter jacket still available for purchase?
Category: new_order



In [32]:
# models = "gemini-2.0-flash"
def classify_message_with_gemini(text):
    prompt = f"""You are an AI assistant that classifies WhatsApp messages into one of the following categories:

1. new_order - Customer shows clear intent to purchase something.
2. order_status - Asks about delivery, tracking, or the progress of an existing order.
3. general_inquiry - Questions about product details, pricing, store hours, payment options, or company policies.
4. complaint - Expressions of dissatisfaction or reports of a bad experience (e.g., broken item, late delivery, faulty service).
5. return_refund - Clear requests for returns, refunds, or exchanges.
6. follow_up - Refers to a previous conversation or nudges for a reply.
7. feedback - Reviews, suggestions, compliments, or general opinions.
8. others - Any message that doesn’t fit the above.

Classify the following message:

"{text}"

Reply with only the category name."""
    
    response = client.models.generate_content(model="gemini-2.0-flash-lite",contents=prompt)
    
    return response.text

    
# sample = messages[57]
# predicted = classify_message_with_gemini(sample["message"])

# print("Text:", sample["message"])
# print("Actual Category:", sample["category"])
# print("Predicted Category:", predicted)
    

In [35]:
import random
results=[]
correct=0
selected_messages =10
for i, msg in enumerate(random.sample(messages, selected_messages), start=1):
    text = msg["message"]
    actual = msg["category"]
    
    response = classify_message_with_gemini(msg)
    predicted = response.strip()
    results.append({
    "message": text,
    "actual": actual,
    "predicted": predicted
    })
    if predicted == actual:
        correct += 1

    print(f"{i:>3}. Text: {text}")
    print(f"   Actual: {actual} | Predicted: {predicted}\n")
    
accuracy = correct / selected_messages
print(f"\n Accuracy: {accuracy:.2%} ({correct}/{selected_messages})")

  1. Text: It’s been a week, still no update on my order.
   Actual: order_status | Predicted: order_status

  2. Text: Is my order out for delivery today?
   Actual: order_status | Predicted: order_status

  3. Text: Awaiting confirmation.
   Actual: follow_up | Predicted: follow_up

  4. Text: Please escalate this.
   Actual: follow_up | Predicted: follow_up

  5. Text: Checking in again on this.
   Actual: follow_up | Predicted: follow_up

  6. Text: Do you offer international shipping?
   Actual: general_inquiry | Predicted: general_inquiry

  7. Text: Want to return the product due to late delivery.
   Actual: return_refund | Predicted: return_refund

  8. Text: Did you receive my payment for order #999?
   Actual: order_status | Predicted: order_status

  9. Text: Your website shows shipped but no updates from the courier.
   Actual: order_status | Predicted: order_status

 10. Text: This is not what I ordered.
   Actual: complaint | Predicted: complaint


 Accuracy: 100.00% (10/